In [1]:
import requests
from skimage import io
import cv2
import json
import os
import pprint

import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

os.sys.path

['',
 '/home/kaijuan/anaconda3/lib/python36.zip',
 '/home/kaijuan/anaconda3/lib/python3.6',
 '/home/kaijuan/anaconda3/lib/python3.6/lib-dynload',
 '/home/kaijuan/anaconda3/lib/python3.6/site-packages',
 '/home/kaijuan/anaconda3/lib/python3.6/site-packages/IPython/extensions',
 '/home/kaijuan/.ipython']

In [2]:
# image feature generation using azure img api and open cv generation


subscription_key_img = '2c5991ae4f584a9b8acd1152cc48da4c'

vision_base_url="https://westcentralus.api.cognitive.microsoft.com/vision/v1.0/"
vision_analyse_url=vision_base_url + 'analyze?'

In [4]:
def extract_high_level_features(image_path):
    headers={'Ocp-Apim-Subscription-Key':subscription_key_img,
             'Content-Type': 'application/json'}
    params = {'visualfeatures':'Adult, Categories, Description, Color, Faces, ImageType, Tags'}
    data = {'url':image_path}

    response = requests.post(vision_analyse_url, headers=headers, params=params, json=data)
    response.raise_for_status()
    data_back = response.json()
    
    features = {
        "clip_art_type"                      : data_back["imageType"]["clipArtType"],
        "line_drawing_type"                  : data_back["imageType"]["lineDrawingType"],
        "is_black_and_white"                 : int(data_back["color"]["isBwImg"]),
        "is_adult_content"                   : int(data_back["adult"]["isAdultContent"]),
        "adult_score"                        : data_back["adult"]["adultScore"],
        "is_racy"                            : int(data_back["adult"]["isRacyContent"]),
        "racy_score"                         : data_back["adult"]["racyScore"],
        "has_faces"                          : int(len(data_back["faces"])),
        "num_faces"                          : len(data_back["faces"]),
        "is_dominant_color_background_black" : int(data_back["color"]["dominantColorBackground"] == "Black"),
        "is_dominant_color_foreground_black" : int(data_back["color"]["dominantColorForeground"] == "Black")
    }
    
    return list(features.values())

In [6]:
width, height = 128, 64
descriptor = cv2.HOGDescriptor(_winSize = (width,height),
                               _blockSize = (16,16),
                               _blockStride = (8,8),
                               _cellSize = (8,8),
                               _nbins = 9)

# compute HOG features for an image
def extract_low_level_features(image_path, descriptor=descriptor):
    image = io.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (width,height))

    hog = descriptor.compute(image)
    hog = hog.flatten().tolist()
    return hog

In [7]:
# conduct low- and high-level feature extraction for images in a directory
def extract_all_features(image_url_list):
    features = []
    for path in range(len(image_url_list)):
        high_level_features = extract_high_level_features(image_url_list[path])
#         low_level_features = extract_low_level_features(image_url_list[path])
        
        features.append(high_level_features)
    
#         features.append({**high_level_features})
    return features

In [9]:
def read_img_url(jsonFile,filesize):
    data = json.load(open(jsonFile))
    server_img_url_list=[]
    for j in range(0,filesize):
        server_img_url = 'https://cvc.ischool.utexas.edu/~dannag/VizWiz/Images/'+ data[j]['image']
        server_img_url_list.append(server_img_url)
    return server_img_url_list

In [10]:
test_img_url_list=read_img_url('test.json',30)
print(test_img_url_list)
test_img_features=extract_all_features(test_img_url_list)
# print(test_img_features)

['https://cvc.ischool.utexas.edu/~dannag/VizWiz/Images/VizWiz_test_000000020000.jpg', 'https://cvc.ischool.utexas.edu/~dannag/VizWiz/Images/VizWiz_test_000000020001.jpg', 'https://cvc.ischool.utexas.edu/~dannag/VizWiz/Images/VizWiz_test_000000020002.jpg', 'https://cvc.ischool.utexas.edu/~dannag/VizWiz/Images/VizWiz_test_000000020003.jpg', 'https://cvc.ischool.utexas.edu/~dannag/VizWiz/Images/VizWiz_test_000000020004.jpg', 'https://cvc.ischool.utexas.edu/~dannag/VizWiz/Images/VizWiz_test_000000020005.jpg', 'https://cvc.ischool.utexas.edu/~dannag/VizWiz/Images/VizWiz_test_000000020006.jpg', 'https://cvc.ischool.utexas.edu/~dannag/VizWiz/Images/VizWiz_test_000000020007.jpg', 'https://cvc.ischool.utexas.edu/~dannag/VizWiz/Images/VizWiz_test_000000020008.jpg', 'https://cvc.ischool.utexas.edu/~dannag/VizWiz/Images/VizWiz_test_000000020009.jpg', 'https://cvc.ischool.utexas.edu/~dannag/VizWiz/Images/VizWiz_test_000000020010.jpg', 'https://cvc.ischool.utexas.edu/~dannag/VizWiz/Images/VizWiz_tes

In [11]:
from pandas import DataFrame
from IPython.display import display

img_df = DataFrame(data=test_img_features)
display(img_df)

,0,1,2,3,4,5,6,7,8,9,10
0,0,0,0,0,0.035585,0,0.025597,0,0,1,1
1,0,0,0,0,0.067857,0,0.084319,0,0,1,0
2,0,0,0,0,0.007775,0,0.009652,0,0,0,0
3,0,0,0,0,0.017246,0,0.014839,0,0,0,1
4,0,0,0,0,0.009939,0,0.013404,0,0,1,0
5,0,0,0,0,0.027363,0,0.030732,0,0,0,0
6,0,0,0,0,0.022667,0,0.039295,0,0,0,0
7,0,0,0,0,0.084678,0,0.145256,0,0,1,0
8,0,0,0,0,0.047701,0,0.063104,0,0,0,0
9,0,0,0,0,0.069645,0,0.043704,0,0,0,0


In [12]:
'''
text feature extration
'''

import nltk
import pprint
import requests
import json
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/kaijuan/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/kaijuan/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /home/kaijuan/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /home/kaijuan/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /home/kaijuan/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /home/kaijuan/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading p

[nltk_data]    |   Package reuters is already up-to-date!
[nltk_data]    | Downloading package rte to /home/kaijuan/nltk_data...
[nltk_data]    |   Package rte is already up-to-date!
[nltk_data]    | Downloading package semcor to
[nltk_data]    |     /home/kaijuan/nltk_data...
[nltk_data]    |   Package semcor is already up-to-date!
[nltk_data]    | Downloading package senseval to
[nltk_data]    |     /home/kaijuan/nltk_data...
[nltk_data]    |   Package senseval is already up-to-date!
[nltk_data]    | Downloading package sentiwordnet to
[nltk_data]    |     /home/kaijuan/nltk_data...
[nltk_data]    |   Package sentiwordnet is already up-to-date!
[nltk_data]    | Downloading package sentence_polarity to
[nltk_data]    |     /home/kaijuan/nltk_data...
[nltk_data]    |   Package sentence_polarity is already up-to-date!
[nltk_data]    | Downloading package shakespeare to
[nltk_data]    |     /home/kaijuan/nltk_data...
[nltk_data]    |   Package shakespeare is already up-to-date!
[nltk_dat

True

In [13]:
subscription_key_text = 'f7107338c32d4fcb9191477c4227e962'

text_analytics_base_url = "https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.0/"
language_api_url = text_analytics_base_url + "languages"
sentiment_api_url = text_analytics_base_url + "sentiment"
key_phrase_api_url = text_analytics_base_url + "keyPhrases"

In [14]:
# evaluate text "documents" using the Microsoft Azure Cognitive Services Text Analytics API
def analyze_text(documents):
    headers   = {"Ocp-Apim-Subscription-Key": subscription_key_text}
    
    # query API for language analysis
    response  = requests.post(language_api_url, headers=headers, json=documents)
    languages = response.json()
    
    # query API for sentiment analysis
    response  = requests.post(sentiment_api_url, headers=headers, json=documents)
    sentiments = response.json()
    
    # query API for key phrases extraction
    response  = requests.post(key_phrase_api_url, headers=headers, json=documents)
    key_phrases = response.json()
    
    return (languages, sentiments, key_phrases)

In [15]:
def pos_tag_text(documents):
    tag_counts = []
    
    for doc in documents['documents']:
        text = nltk.word_tokenize(doc['text'])
        tags = nltk.pos_tag(text)
        frequencies = nltk.FreqDist(tag for (word, tag) in tags)
        tag_count={
            'ADJ':0,
            'ADV':0,
            'CONJ':0,
            'NOUN':0,
            'NUM':0,
            'PRON':0,
            'VERB':0
        }
        
        for tag, count in frequencies.most_common():
            tag_count[tag] =count
        tag_counts.append(tag_count)
    return tag_counts


In [18]:
# get the question documents from the training set
def get_test_questions(json_file,filesize):
#     labels = json.load(open('train.json'))
    labels = json.load(open(json_file))
    documents = []
    for index, document in enumerate(labels):
        if index < filesize:
            documents.append({
                'id': index,
                'text': document['question']
            })
    
    return {'documents': documents}

In [19]:

test_documents = get_test_questions('test.json',30)
pprint.pprint(test_documents)

{'documents': [{'id': 0, 'text': 'What is this? And what color is it?'},
               {'id': 1, 'text': 'What is this?'},
               {'id': 2,
                'text': 'Has this oven gotten up to four hundred fifty degrees '
                        'Fahrenheit yet?'},
               {'id': 3, 'text': 'What is this?'},
               {'id': 4, 'text': 'What is this?'},
               {'id': 5, 'text': 'What kind of key is this?'},
               {'id': 6, 'text': 'What does it say on here?'},
               {'id': 7, 'text': 'What is this? '},
               {'id': 8, 'text': 'What is this? What is this? '},
               {'id': 9,
                'text': 'Do these beans look like black beans or pinto beans?'},
               {'id': 10, 'text': 'Whats the date?'},
               {'id': 11, 'text': 'what is this?'},
               {'id': 12, 'text': 'What is this game?'},
               {'id': 13, 'text': 'This piece of paper.'},
               {'id': 14, 'text': 'What brand is thi

In [24]:
def get_tfidf_doc(filesize):
    labels = json.load(open('test.json'))
    for ele in labels:
        tfidf_doc = []
        for index, document in enumerate(labels):
            if index < filesize:
                tfidf_doc.append(document['question'])
    return tfidf_doc

In [31]:
'''
get tfidf
'''
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_doc = get_tfidf_doc(30)
vectorizer = TfidfVectorizer(stop_words='english')

tfidf_test = vectorizer.fit_transform(tfidf_doc)
# print(tfidf_test)
# tfidf_train.shape
# print(tfidf_train.shape)

tfidf_test_array=tfidf_test.toarray()
tfidf_df = DataFrame(data=tfidf_test_array)
display(tfidf_df)


,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
0,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.83869,0.0,0.279563,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [28]:
languages, sentiments, key_phrases = analyze_text(test_documents)

languages, sentiments, key_phrases = languages['documents'], sentiments['documents'], key_phrases['documents']

tag_counts = pos_tag_text(test_documents)
tag_features=[]

for tag in tag_counts:
    temp_list=[]
    for i in tag:
        temp_list.append(tag[i])
    tag_features.append(temp_list)
print(len(tag_features[0]))

tag_df=DataFrame(data=tag_features)
tag_df=tag_df.replace(np.nan,0)
display(tag_df)

14


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0,0,0,0,0,0,0,2,2,2,1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,0,0,0,1,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,0,0,0,0,2,2,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
3,0,0,0,0,0,0,0,1,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,0,0,0,1,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0,0,0,0,0,0,0,2,1,1,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0,0,0,0,0,0,0,1,1,1,1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
7,0,0,0,0,0,0,0,1,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0,0,0,0,0,0,0,2,2,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0,0,0,0,0,0,0,3,1,1,1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0


In [37]:
import pandas as pd
azure_features=[]

for i in range(len(test_documents['documents'])):
    analysis=[]
    analysis.append(sentiments[i]['score'])
    analysis.append(len(key_phrases[i]['keyPhrases']))
    azure_features.append(analysis)
    
azure_df=DataFrame(data=azure_features)
# display(azure_df)
test_features_df=pd.concat([azure_df,tag_df,tfidf_df,img_df], axis=1, ignore_index=True)
display(test_features_df)

test_features_df.to_csv("test_features.csv",encoding='utf-8',index=False)

    


,0,1,2,3,4,5,6,7,8,9,...,71,72,73,74,75,76,77,78,79,80
0,0.748632,1,0,0,0,0,0,0,0,2,...,0,0,0,0.035585,0,0.025597,0,0,1,1
1,0.210352,0,0,0,0,0,0,0,0,1,...,0,0,0,0.067857,0,0.084319,0,0,1,0
2,0.500000,2,0,0,0,0,0,0,0,2,...,0,0,0,0.007775,0,0.009652,0,0,0,0
3,0.210352,0,0,0,0,0,0,0,0,1,...,0,0,0,0.017246,0,0.014839,0,0,0,1
4,0.210352,0,0,0,0,0,0,0,0,1,...,0,0,0,0.009939,0,0.013404,0,0,1,0
5,0.765828,1,0,0,0,0,0,0,0,2,...,0,0,0,0.027363,0,0.030732,0,0,0,0
6,0.696522,0,0,0,0,0,0,0,0,1,...,0,0,0,0.022667,0,0.039295,0,0,0,0
7,0.210352,0,0,0,0,0,0,0,0,1,...,0,0,0,0.084678,0,0.145256,0,0,1,0
8,0.210352,0,0,0,0,0,0,0,0,2,...,0,0,0,0.047701,0,0.063104,0,0,0,0
9,0.500000,2,0,0,0,0,0,0,0,3,...,0,0,0,0.069645,0,0.043704,0,0,0,0
